In [ ]:
require 'torch'
require 'gnuplot'
require 'rnn'
require 'image'
Plot = require 'itorch.Plot'

In [ ]:
-- scatter plots
plot = Plot():circle(torch.range(1, #err), 
                err, 'red', 'RMSE'):draw()
plot:line(torch.range(1, #err), 
                err, 'red', ''):redraw()
plot:title('Plot of Loss Function'):redraw()
plot:xaxis('Iterations'):yaxis('Mean Squared Error'):redraw()
plot:legend(true)
plot:redraw()